In [8]:
!pip install Bani
import sys
from typing import List,Tuple,Dict

In [2]:
# appending paths as this is a jupyter notebook, if you want to use from the source code !!!
"""
sys.path.append('./')
sys.path.append('./core')
"""


"\nsys.path.append('./')\nsys.path.append('./core')\n"

In [2]:
from Bani.Bani import Bani
from Bani.core.FAQ import FAQ

In [3]:
"""
This cell is optional, we read the data from any faq here
In this case we use the orignal faq pickle files , you may use
csv or any other format !!!
"""
import pickle
def save_dict(obj, path):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_dict(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def extractQA(orignalDct):
    """
    Just returns the questions and answers in a list , 
    NOthing fancy
    """
    q2L = orignalDct["question_to_label"]
    a2L = orignalDct["answer_to_label"]
    

    l2A = dict()
    for a,l in a2L.items():
        l2A[l] = a


    questions = []
    answers = []

    for q,l in q2L.items():
        questions.append(q)
        answers.append(l2A[l])
    return questions , answers
    

babyBonusOrignal = load_dict("./tutorialFAQs/baby_bonus_orignal.pkl") 
comcareOrignal =  load_dict("./tutorialFAQs/comcare_orignal.pkl")


b_questions , b_answers = extractQA(babyBonusOrignal)
c_questions , c_answers = extractQA(comcareOrignal)



    


In [12]:
"""
NOw the real work starts , We have question answer pairs from 
baby bonus and comcare , We need to create a new faq for both of them

NOTE - This is the starting point , we need a LIst of questions and a list of answers
THE LENGTH OF BOTH LIST MUST BE SAME
if two questions answer the same questions then just copy the answer 
"""


# first we convert the questions and answers to 
babyBonusFAQ = FAQ(name = "babyBonus",questions = b_questions, answers = b_answers)
comcareFAQ = FAQ(name = "comcare", questions = c_questions , answers = c_answers)

In [6]:
"""
Until this point we only have the orignal questions and answers in the faq
We now need to do two things
1) generate questions using pipelined
2) assign vectors to the questions using a model  


you can use your own generation pipeline , or use the default pipeline provided !!!
if you choose to implement your own pipeline make sure that your class implements at least one of 
batch_generate or exact_batch_generate methods


"""
from Bani.core.defaults import defaultGenerateManager


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
"""
This is an example of creating your own GenerateManager

"""

from Bani.core.generation import GenerateManager

class MyProducer1:
    def __init__(self):
        pass
    
    def batch_generate(questions : List[str]) -> Dict[str, List[str]]:
        """
        Takes list of questions and returns a dict , with each question 
        mapped to the list of generated questions
        """
        
        resultDict = dict()
        for question in questions:
            resultDict[question] = ["generated1", "generated2", "and so on"]
        
        return resultDict
    
class MyProducer2:
    def __init__(self):
        pass
    
    def batch_generate(questions : List[str]) -> Dict[str, List[str]]:
        """
        Takes list of questions and returns a dict , with each question 
        mapped to the list of generated questions
        """
        
        resultDict = dict()
        for question in questions:
            resultDict[question] = ["generated1", "generated2", "and so on"]
        
        return resultDict
    
    
names = ["myProducer1_name", "myProducer2_name"]
toGenerate = [3,5] # At max generate 3 for first producer and 5 for second
producers = [MyProducer1(), MyProducer2()]


myGenerateManager = GenerateManager(producers = producers , names = names , nums = toGenerate)
    

    

    
    

    

In [13]:
"""

Now we will build the FAQ using the generationManager

"""

babyBonusFAQ.buildFAQ(generator = defaultGenerateManager)

working with SymSub pipeline
Initializing spaCy model...


  0%|                                                                                          | 0/291 [00:00<?, ?it/s]

working with FPM pipeline


 15%|████████████▎                                                                   | 45/291 [00:00<00:00, 445.55it/s]

working with EDA pipeline


  0%|                                                                                          | 0/291 [00:00<?, ?it/s]

working with nlpAug pipeline


100%|████████████████████████████████████████████████████████████████████████████████| 291/291 [02:17<00:00,  2.11it/s]


In [14]:
comcareFAQ.buildFAQ(generator  = defaultGenerateManager)

  0%|                                                                                          | 0/288 [00:00<?, ?it/s]

working with SymSub pipeline


  0%|                                                                                          | 0/288 [00:00<?, ?it/s]

working with FPM pipeline


 27%|█████████████████████▉                                                          | 79/288 [00:00<00:00, 399.13it/s]

working with EDA pipeline


  0%|                                                                                          | 0/288 [00:00<?, ?it/s]

working with nlpAug pipeline


100%|████████████████████████████████████████████████████████████████████████████████| 288/288 [02:53<00:00,  1.66it/s]


In [15]:
"""
Now we can save the FAQ at any time we want !! , just pass the path to the root dir 
"""
babyBonusFAQ.save("./faqStore")
comcareFAQ.save("./faqStore")

TypeError: expected str, bytes or os.PathLike object, not tuple

In [ ]:
"""
The saved FAQ can be easily loaded 
"""
babyBonusFAQ = FAQ("babyBonus")
babyBonusFAQ.load("./faqStore")

In [ ]:
comcareFAQ = FAQ("comcare")
comcareFAQ.load("./faqStore")

In [ ]:
"""
Lets now take a look at the bot part
"""

bot = ChadBot(FAQs =[babyBonusFAQ, comcareFAQ], modelPath = None)

In [ ]:
"""
We can also save the FAQ again to preserve the assigned vectors ,
The next time you load these FAQs with the bot the vectors will not be reassigned ,
You need to clear the assigned vectors if you want to reassign
"""
bot.saveFAQs("./faqStore")


In [ ]:
out = bot.findClosest("what is comcare?", K = 5)

In [ ]:
bot.findClosestFromFAQ(1,"What is comcare?", K = 1, topSimilar= 5).answer.text

In [8]:
"""
Training
"""

'\nTraining\n'

In [ ]:
bot.train("./checkpoints/dummy")